In [ ]:
import backtrader as bt
#import yfinance as yf
import pandas as pd
#import pyfolio as pf
import datetime
import math
# import the package after installation
from backtrader_plotly.plotter import BacktraderPlotly
from backtrader_plotly.scheme import PlotScheme
import plotly.io
import warnings
import backtrader.feeds as btfeeds
warnings.filterwarnings("ignore")

import backtrader.indicators as btind # 导入策略分析模块
import numpy as np
import backtrader as bt
import plotly.graph_objects as go

## 策略逻辑

In [ ]:


class RollingBiasQuantile(bt.Indicator):
    lines = ('quantile', )
    params = (('bias_len', 30), ('quantile', 50))

    def __init__(self):
        self.addminperiod(self.p.bias_len)


    def next(self):
        vals = self.data.get(size=self.p.bias_len)
        # print('p.quantile: ', self.p.quantile)
        self.lines.quantile[0] = np.percentile(vals, self.p.quantile)




class BiasStrategy(bt.Strategy):
    # 不要更改这里的值，要更改下面main的值！！
    params = (("para_ma", 10),
              ("bias_len",200),
              ("para_risk_ma", 5),
              ("high_q", 0.8),
              ("low_q", 0.2),
              ("bias_thresh", 0.001),
              )

    def __init__(self):
        self.order_history = []
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # get all indicators
        print("***printing: self.datas[0].lines.getlinealiases():", self.datas[0].lines.getlinealiases())
        # print("***printing: date", self.datas[0].datetime.date(0))
        print("***printing close:", self.datas[0].close[0])
        print("***printing the past N values of a certain col：", self.datas[0].close.get(ago=-1, size = 3))

        # get close
        self.close_price = btind.SMA(self.datas[0].close, period=1)
        # calculate ma
        self.ma = btind.SMA(self.datas[0].close, period=self.p.para_ma)
        # calculate bias
        self.bias = self.close_price/self.ma
        # print('printing bias: ', self.bias)
        # calculate rolling bias quantiles and bias_hi and bias_lo
        self.bias_hi = RollingBiasQuantile(self.bias, bias_len = self.p.bias_len, quantile=self.p.high_q)
        self.bias_lo = RollingBiasQuantile(self.bias, bias_len = self.p.bias_len, quantile=self.p.low_q)
        # calculate risk_ma
        self.risk_ma = btind.SMA(self.datas[0].close, period=self.p.para_risk_ma)

        self.dataclose = self.datas[0].close
        self.data_open = self.datas[0].open
        self.data_high = self.datas[0].high
        self.data_low = self.datas[0].low
        self.data_volume = self.datas[0].volume
        self.dates = self.datas[0].datetime
        self.data_list = []

    def next(self):

        if self.order:
            return
        print("------------------")
        print(self.datas[0].datetime.datetime(0))

        self.data_list.append([self.dates.datetime(0), 
                self.data_open[0], 
                self.data_high[0], 
                self.data_low[0], 
                self.dataclose[0], 
                self.data_volume[0], 
                self.bias[0], 
                self.bias_hi[0], 
                self.bias_lo[0], 
                self.risk_ma[0]])
        
        # create signals:
        # self.short_signal = (self.bias[0]>self.bias_hi[0])&(self.bias[0]-1>self.p.bias_thresh)&(self.close_price[0]<self.risk_ma[0])
        # self.long_signal = (self.bias[0]<self.bias_lo[0])&(1-self.bias[0]>self.p.bias_thresh)&(self.close_price[0]>self.risk_ma[0])
        self.short_signal = (self.bias[0]>self.bias_hi[0])&(self.bias[0]-1>self.p.bias_thresh)&(self.close_price[0]<self.risk_ma[0])
        self.long_signal = (self.bias[0]<self.bias_lo[0])&(1-self.bias[0]>self.p.bias_thresh)&(self.close_price[0]>self.risk_ma[0])

        self.close_long = self.close_price[0]<self.risk_ma[0]
        self.close_short = self.close_price[0]>self.risk_ma[0]
        # print(self.close_long)
        # print(self.close_short)
        print(self.short_signal)
        print(self.long_signal)
        
        print("close vs risk_ma", self.close_price[0], self.risk_ma[0])


        if self.position.size > 0: # if there're long position
            if self.close_long: # and there's risk
                self.order = self.sell() # close long pos
                print("已下单关仓多头")

        elif self.position.size < 0: # if there're short position
            if self.close_short: # and there's risk
                self.order = self.buy() # close short pos
                print("已下单关仓空头")

        elif self.position.size == 0: # if there're no positions
            if self.short_signal: # if short signals are triggered
                self.order = self.sell() # enter short pos
                print("已下单做空")

            elif self.long_signal: # if long signals are triggered
                self.order = self.buy() # enter long pos
                print("已下单做多")

                


        # print('printing time: ', self.datas[0].datetime.datetime(0))
        # print('printing close price', self.close_price[0])
        # print('printing last bias: ', self.bias[0])
        # print('bias hi: ', self.bias_hi[0])
        # print('bias lo: ', self.bias_lo[0])

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                        order.executed.value,
                        order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                        order.executed.value,
                        order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None



    def stop(self):
        self.data_df = pd.DataFrame(self.data_list, columns=['datetime', 'open', 'high', 'low', 'close', 
                                                             'volume', 'bias', 'bias_hi', 'bias_lo', 'risk_ma'])
        self.data_df.set_index('datetime', inplace=True)

## 策略优化 - 寻找相对最优参数

In [ ]:
if __name__ == '__main__':
    # 1. 实例化大脑
    cerebro_opt= bt.Cerebro()

    # 2. 添加/优化策略
    cerebro_opt.optstrategy(BiasStrategy, 
                            para_ma = range(10, 30, 2),
                            bias_len = range(50, 300, 50), 
                            para_risk_ma = 5,
                            high_q = 80,
                            low_q = 20,
                            bias_thresh=0.0005
                            )
    # 3. 加载/添加数据
    data_opt = bt.feeds.GenericCSVData(
        dataname='C:/Users/bradsun/workspace/prop_investing/futures_data/a2307.csv',
        timeframe=btfeeds.TimeFrame.Minutes,
        fromdate=datetime.datetime(2023, 5, 18),
        todate=datetime.datetime(2023, 6, 1),
        nullvalue=0.0,
        dtformat=('%Y/%m/%d %H:%M'),
        datetime=2,
        high=4,
        low=5,
        open=3,
        close=6,
        volume=7,
        openinterest=-1
    )
    cerebro_opt.adddata(data_opt)

    # 4. 交易与账户设置
    # cerebro_opt.addsizer(bt.sizers.SizerFix, stake=1)
    cerebro_opt.addsizer(bt.sizers.PercentSizer, percents=20)
    cerebro_opt.broker.setcash(1000000)
    cerebro_opt.broker.setcommission(commission=0.0002)
    cerebro_opt.broker.set_slippage_perc(0.0000)


    # 5. 添加分析指标
    # 返回总收益率
    cerebro_opt.addanalyzer(bt.analyzers.Returns, _name='_TotalReturns')
    # 返回年初至年末的年度收益率
    cerebro_opt.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')
    # 计算最大回撤相关指标
    cerebro_opt.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')
    # 计算年化收益
    cerebro_opt.addanalyzer(bt.analyzers.Returns, _name='_Returns', tann=252)
    # 计算年化夏普比率
    cerebro_opt.addanalyzer(bt.analyzers.SharpeRatio_A, _name='_SharpeRatio_A', 
                            timeframe=bt.TimeFrame.Minutes, riskfreerate=0.01)
    # 返回收益率时序
    cerebro_opt.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')

    # 6. 启动回测
    results = cerebro_opt.run(maxcpus=1)  

In [ ]:
# 7. 建立记录优化结果的数据表并展示
def get_my_analyzer(results):
    analyzer = {}
    # 返回参数
    analyzer['para_ma'] = results.p.para_ma
    analyzer['bias_len'] = results.p.bias_len
    analyzer['para_risk_ma'] = results.p.para_risk_ma
    analyzer['high_q'] = results.p.high_q
    analyzer['low_q'] = results.p.low_q
    analyzer['bias_thresh'] = results.p.bias_thresh
    # 提取年化收益
    analyzer['回测期总收益率'] = results.analyzers._TotalReturns.get_analysis()['rtot']
    analyzer['年化收益率'] = results.analyzers._Returns.get_analysis()['rnorm']
    analyzer['年化收益率（%）'] = results.analyzers._Returns.get_analysis()['rnorm100']
    # 提取最大回撤
    analyzer['最大回撤（%）'] = results.analyzers._DrawDown.get_analysis()['max']['drawdown'] * (-1)
    # 提取夏普比率
    analyzer['年化夏普比率'] = results.analyzers._SharpeRatio_A.get_analysis()['sharperatio']
    return analyzer

ret = []
for i in results:
    ret.append(get_my_analyzer(i[0])) # 注意：这里i[0]和优化时的写法不一样
pd.DataFrame(ret).sort_values(by='年化收益率', ascending=False)

## 可视化用于计算信号的时间序列值，以此检查计算的正确与否

In [ ]:
class RollingBiasQuantile(bt.Indicator):
    lines = ('quantile', )
    params = (('bias_len', 30), ('quantile', 50))

    def __init__(self):
        self.addminperiod(self.p.bias_len)


    def next(self):
        vals = self.data.get(size=self.p.bias_len)
        # print('p.quantile: ', self.p.quantile)
        self.lines.quantile[0] = np.percentile(vals, self.p.quantile)

class BiasStrategy(bt.Strategy):
    # 不要更改这里的值，要更改下面main的值！！
    params = (("para_ma", 10),
              ("bias_len",200),
              ("para_risk_ma", 5),
              ("high_q", 0.8),
              ("low_q", 0.2),
              ("bias_thresh", 0.001),
              )

    def __init__(self):
        self.order_history = []
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # get all indicators
        print("***printing: self.datas[0].lines.getlinealiases():", self.datas[0].lines.getlinealiases())
        # print("***printing: date", self.datas[0].datetime.date(0))
        print("***printing close:", self.datas[0].close[0])
        print("***printing the past N values of a certain col：", self.datas[0].close.get(ago=-1, size = 3))

        # get close
        self.close_price = btind.SMA(self.datas[0].close, period=1)
        # calculate ma
        self.ma = btind.SMA(self.datas[0].close, period=self.p.para_ma)
        # calculate bias
        self.bias = self.close_price/self.ma
        # print('printing bias: ', self.bias)
        # calculate rolling bias quantiles and bias_hi and bias_lo
        self.bias_hi = RollingBiasQuantile(self.bias, bias_len = self.p.bias_len, quantile=self.p.high_q)
        self.bias_lo = RollingBiasQuantile(self.bias, bias_len = self.p.bias_len, quantile=self.p.low_q)
        # calculate risk_ma
        self.risk_ma = btind.SMA(self.datas[0].close, period=self.p.para_risk_ma)

        self.dataclose = self.datas[0].close
        self.data_open = self.datas[0].open
        self.data_high = self.datas[0].high
        self.data_low = self.datas[0].low
        self.data_volume = self.datas[0].volume
        self.dates = self.datas[0].datetime
        self.data_list = []

    def next(self):

        if self.order:
            return
        print("------------------")
        print(self.datas[0].datetime.datetime(0))

        self.data_list.append([self.dates.datetime(0), 
                self.data_open[0], 
                self.data_high[0], 
                self.data_low[0], 
                self.dataclose[0], 
                self.data_volume[0], 
                self.bias[0], 
                self.bias_hi[0], 
                self.bias_lo[0], 
                self.risk_ma[0]])
        
        # create signals:
        # self.short_signal = (self.bias[0]>self.bias_hi[0])&(self.bias[0]-1>self.p.bias_thresh)&(self.close_price[0]<self.risk_ma[0])
        # self.long_signal = (self.bias[0]<self.bias_lo[0])&(1-self.bias[0]>self.p.bias_thresh)&(self.close_price[0]>self.risk_ma[0])
        self.short_signal = (self.bias[0]>self.bias_hi[0])&(self.bias[0]-1>self.p.bias_thresh)&(self.close_price[0]<self.risk_ma[0])
        self.long_signal = (self.bias[0]<self.bias_lo[0])&(1-self.bias[0]>self.p.bias_thresh)&(self.close_price[0]>self.risk_ma[0])

        self.close_long = self.close_price[0]<self.risk_ma[0]
        self.close_short = self.close_price[0]>self.risk_ma[0]
        # print(self.close_long)
        # print(self.close_short)
        print(self.short_signal)
        print(self.long_signal)
        
        print("close vs risk_ma", self.close_price[0], self.risk_ma[0])


        if self.position.size > 0: # if there're long position
            if self.close_long: # and there's risk
                self.order = self.sell() # close long pos
                print("已下单关仓多头")

        elif self.position.size < 0: # if there're short position
            if self.close_short: # and there's risk
                self.order = self.buy() # close short pos
                print("已下单关仓空头")

        elif self.position.size == 0: # if there're no positions
            if self.short_signal: # if short signals are triggered
                self.order = self.sell() # enter short pos
                print("已下单做空")

            elif self.long_signal: # if long signals are triggered
                self.order = self.buy() # enter long pos
                print("已下单做多")

                


        # print('printing time: ', self.datas[0].datetime.datetime(0))
        # print('printing close price', self.close_price[0])
        # print('printing last bias: ', self.bias[0])
        # print('bias hi: ', self.bias_hi[0])
        # print('bias lo: ', self.bias_lo[0])

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                        order.executed.value,
                        order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                        order.executed.value,
                        order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None



    def stop(self):
        self.data_df = pd.DataFrame(self.data_list, columns=['datetime', 'open', 'high', 'low', 'close', 
                                                             'volume', 'bias', 'bias_hi', 'bias_lo', 'risk_ma'])
        self.data_df.set_index('datetime', inplace=True)

In [ ]:
if __name__ == '__main__':
    # 0. 参数设置
    para_ma = 18
    bias_len = 100
    para_risk_ma = 5
    high_q = 80
    low_q = 20
    bias_thresh=0.0005

    # 1. 实例化大脑
    cerebro = bt.Cerebro()
    # 2. 添加/优化策略
    cerebro.addstrategy(BiasStrategy, para_ma=para_ma, 
                                        bias_len=bias_len,
                                        para_risk_ma=para_risk_ma,
                                        high_q=high_q,
                                        low_q=low_q,
                                        bias_thresh=bias_thresh
                                        )
    # 3. 加载/添加数据
    data = bt.feeds.GenericCSVData(
        dataname='C:/Users/bradsun/workspace/prop_investing/futures_data/a2307.csv',
        timeframe=btfeeds.TimeFrame.Minutes,
        fromdate=datetime.datetime(2023, 5, 18),
        todate=datetime.datetime(2023, 6, 1),
        nullvalue=0.0,
        dtformat=('%Y/%m/%d %H:%M'),
        datetime=2,
        high=4,
        low=5,
        open=3,
        close=6,
        volume=7,
        openinterest=-1
    )
    cerebro.adddata(data) #if using resample, then comment this line of code    

    # 4. 交易与账户设置
    # cerebro.addsizer(bt.sizers.SizerFix, stake=1)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=20)
    cerebro.broker.setcash(1000000)
    cerebro.broker.setcommission(commission=0.0002)
    cerebro.broker.set_slippage_perc(0.0000)

    # 5. 添加分析指标
    # 返回总收益率
    cerebro.addanalyzer(bt.analyzers.Returns, _name='_TotalReturns')
    # 返回年初至年末的年度收益率
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')
    # 计算最大回撤相关指标
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')
    # 计算年化收益
    cerebro.addanalyzer(bt.analyzers.Returns, _name='_Returns', tann=252)
    # 计算年化夏普比率
    cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='_SharpeRatio_A', 
                            timeframe=bt.TimeFrame.Minutes, riskfreerate=0.01)
    # 返回收益率时序
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')
    # 交易分析
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")

    # 6. 启动回测
    results = cerebro.run()

    # 7. 记录结果
    thestrat = results[0]
    
    trades = thestrat.analyzers.ta.get_analysis()
    scheme = PlotScheme(decimal_places=2, max_legend_text_width=25)
    figs = cerebro.plot(BacktraderPlotly(show=False, scheme=scheme))

    # 8. 绘制回测可视化结果1/2
    for i, each_run in enumerate(figs):
        each_run[i].update_layout(xaxis=dict(type='category'))
        for j, each_strategy_fig in enumerate(each_run):
            each_strategy_fig.update_layout(xaxis=dict(type='category'))
            each_strategy_fig.show()
            # open plot in browser
            # save the html of the plot to a variable
            html = plotly.io.to_html(each_strategy_fig, full_html=False)
            # write html to disk
            plotly.io.write_html(each_strategy_fig, f'{i}_{j}.html', full_html=True) 


In [ ]:
# 9. 绘制回测可视化结果1/2
df = thestrat.data_df
fig = go.Figure()
fig.add_trace(go.Candlestick(x=df.index,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            yaxis='y2'))

fig.add_trace(go.Scatter(x=df.index, y=df['bias'], mode='lines', name='bias'))
fig.add_trace(go.Scatter(x=df.index, y=df['bias_hi'], mode='lines', name='bias_hi'))
fig.add_trace(go.Scatter(x=df.index, y=df['bias_lo'], mode='lines', name='bias_lo'))
fig.add_trace(go.Scatter(x=df.index, y=df['risk_ma'], mode='lines', name='risk_ma', yaxis='y2'))
fig.update_layout(xaxis=dict(type='category'), width=1200,height=1000)
fig.update_layout(yaxis=dict(title='bias'))
fig.update_layout(yaxis2=dict(title='OHLC and risk_ma',overlaying='y',side='right'))
fig.show()

In [ ]:
# 10. 打印回测表现统计数据
ret = []
for i in results:
    ret.append(get_my_analyzer(i)) # 注意：这里i和优化时的写法不一样
pd.DataFrame(ret).sort_values(by='年化收益率', ascending=False)

,para_ma,bias_len,para_risk_ma,high_q,low_q,bias_thresh,回测期总收益率,年化收益率,年化收益率（%）,最大回撤（%）,年化夏普比率
0,18,100,5,80,20,0.0005,0.001263,0.000113,0.011276,-0.18489,-228.298637


In [ ]:
# 11. 交易与订单记录分析
print("Total Trades:",trades['total']['total'],"| Still Opened Trades:",trades['total']['open'],"| Closed Trades:",trades['total']['closed'])
print("Streak Won - Current", trades['streak']['won']['current'], "| Streak Won - Longest", trades['streak']['won']['longest'])
print("Streak Lost Current", trades['streak']['lost']['current'], "| Streak Won Longest", trades['streak']['lost']['longest'])
print("Pnl Gross Total:", trades['pnl']['gross']['total'],"| Pnl Gross Avg:",trades['pnl']['gross']['average'])
print("Pnl Net Total:", trades['pnl']['net']['total'],"| Pnl Net Avg:",trades['pnl']['net']['average'])

Total Trades: 72 | Still Opened Trades: 0 | Closed Trades: 72
Streak Won - Current 0 | Streak Won - Longest 6
Streak Lost Current 3 | Streak Won Longest 4
Pnl Gross Total: 60087.80809610484 | Pnl Gross Avg: 834.5528902236783
Pnl Net Total: 60087.80809610484 | Pnl Net Avg: 834.5528902236783
